In [1]:
using QuantEcon

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
@everywhere using QuantEcon

In [4]:
nprocs()

5

In [5]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [6]:
@everywhere function lucas_tree_spec_rad(;beta=0.96,
                        gamma=2.0,
                        sigma=0.1,
                        b=0.0,
                        rho=0.9,
                        n=200)

    mc = tauchen(n, rho, sigma, b)  
    s = mc.state_values
    J = mc.p .* exp.((1 - gamma) * s)
    return beta * maximum(abs, eigvals(J))
end

In [7]:
lucas_tree_spec_rad(b=0.01)

1.2174480318956242

In [8]:
b_vals = collect(linspace(0.0, 0.5, 500));

In [9]:
out = Array{Float64}(length(b_vals))

@time for i in eachindex(b_vals)
    out[i] = lucas_tree_spec_rad(b=b_vals[i])
end

 50.742339 seconds (24.00 k allocations: 646.454 MiB, 0.09% gc time)


In [10]:
out_shared = SharedArray{Float64}(length(b_vals))

@time @sync @parallel for i in eachindex(b_vals)
    out_shared[i] = lucas_tree_spec_rad(b=b_vals[i])
end

 13.415171 seconds (282.34 k allocations: 15.401 MiB, 0.05% gc time)


4-element Array{Future,1}:
 Future(2, 1, 23, #NULL)
 Future(3, 1, 24, #NULL)
 Future(4, 1, 25, #NULL)
 Future(5, 1, 26, #NULL)

In [11]:
out .== out_shared

500-element BitArray{1}:
  true
 false
  true
 false
 false
 false
  true
  true
  true
 false
  true
  true
 false
     ⋮
  true
 false
 false
  true
 false
 false
  true
 false
  true
  true
 false
 false

In [12]:
out[1:10]

10-element Array{Float64,1}:
 1.34549
 1.33207
 1.31879
 1.30564
 1.29263
 1.27974
 1.26698
 1.25435
 1.24184
 1.22946

In [13]:
out_shared[1:10]

10-element Array{Float64,1}:
 1.34549
 1.33207
 1.31879
 1.30564
 1.29263
 1.27974
 1.26698
 1.25435
 1.24184
 1.22946